In [1]:
import os
import mmcv
import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

/opt/conda/envs/mmseg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# epoch 지정해주기
epoch = 'epoch_35'
# work dir 설정해주기
work_dir = '/opt/ml/input/mmsegmentation/work_dirs/HRNet'

# config file 들고오기
# config file은 수정해서 사용
cfg = Config.fromfile('/opt/ml/input/mmsegmentation/work_dirs/HRNet/HRNet.py')
root='/opt/ml/input/data/images/test'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

/opt/ml/input/mmsegmentation/mmseg/datasets/pipelines/test_time_aug.py:95: UserWarning: flip_direction has no effect when flip is set to False
  warnings.warn(
2023-01-04 01:06:13,821 - mmseg - INFO - Loaded 819 images


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

/opt/ml/input/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


load checkpoint from local path: /opt/ml/input/mmsegmentation/work_dirs/HRNet/epoch_35.pth


In [5]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 819/819, 1.0 task/s, elapsed: 855s, ETA:     0s

In [15]:
model

MMDataParallel(
  (module): EncoderDecoder(
    (backbone): HRNet(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3

In [17]:
# submission 양식에 맞게 output 후처리
input_size = 512
output_size = 256

submission = pd.read_csv("/opt/ml/input/code/submission/sample_submission.csv", index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")

with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)

# PredictionString
for img_id, pred in enumerate(output):

    img_id = datas["images"][img_id]
    file_name = img_id["file_name"]

    temp_mask = []
    pred = pred.reshape(1, 512, 512)
    mask = pred.reshape((1, output_size, input_size//output_size, output_size, input_size//output_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)
    string = oms.flatten()
    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   ,ignore_index=True)


submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)

In [18]:
submission.head()

,image_id,PredictionString
0,batch_01_vt/0021.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0028.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0031.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0032.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0076.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [ ]:
import os
import mmcv
import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel
from argparse import ArgumentParser

import pandas as pd
import numpy as np
import json
import numpy as np
import pydensecrf.densecrf as dcrf
import pydensecrf.utils as utils

class DenseCRF(object):
    def __init__(self, iter_max, pos_w, pos_xy_std, bi_w, bi_xy_std, bi_rgb_std):
        self.iter_max = iter_max
        self.pos_w = pos_w
        self.pos_xy_std = pos_xy_std
        self.bi_w = bi_w
        self.bi_xy_std = bi_xy_std
        self.bi_rgb_std = bi_rgb_std

    def __call__(self, image, probmap):
        C, H, W = probmap.shape

        U = utils.unary_from_softmax(probmap)
        U = np.ascontiguousarray(U)

        image = np.ascontiguousarray(image)

        d = dcrf.DenseCRF2D(W, H, C)
        d.setUnaryEnergy(U)
        d.addPairwiseGaussian(sxy=self.pos_xy_std, compat=self.pos_w)
        d.addPairwiseBilateral(
            sxy=self.bi_xy_std, srgb=self.bi_rgb_std, rgbim=image, compat=self.bi_w
        )

        Q = d.inference(self.iter_max)
        Q = np.array(Q).reshape((C, H, W))

        return Q

# epoch 지정해주기
epoch = 'epoch_35'
# work dir 설정해주기
work_dir = '/opt/ml/input/mmsegmentation/work_dirs/HRNet'

# config file 들고오기
# config file은 수정해서 사용
cfg = Config.fromfile('/opt/ml/input/mmsegmentation/work_dirs/HRNet/HRNet.py')
root='/opt/ml/input/data/images/test'

# dataset config 수정
cfg.data.test.img_dir = root
# cfg.data.test.pipeline[1]['img_scale'] = (512,512)
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir
cfg.model.train_cfg = None

dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'{epoch}.pth')

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

output = single_gpu_test(model, data_loader)